In [ ]:
# 假设我们有50个数据
target_sum = 6568000  # 目标值

最优的一份

In [9]:
import pandas as pd
from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt

# 读取数据
file_path = "C:/Users/海棠微雨/Desktop/cumcm2021c/C/前50家商家供货量(2).xlsx"
data = pd.read_excel(file_path)
supplies = list(data['5年总产值'])
path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/附件1 近5年402家供应商的相关数据.xlsx"
sales = pd.read_excel(path, sheet_name='供应商的供货量（m³）').iloc[:, :-2]

# 定义适应度函数
def evalSupply(individual, supplies):
     total_sum = sum(ind * supply for ind, supply in zip(individual, supplies))
    
     # 如果总和小于600万，返回一个高惩罚值以降低该个体的适应度
     if total_sum < 6200000:
        return 1000000 + (6400000 - total_sum),
    
     # 返回选择的数目的个数作为适应度值（越少越好）
     return sum(individual),

# 定义遗传算法
def genetic_algorithm(supplies):
    # 初始化DEAP的基本设置
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    
    # 二进制编码
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(supplies))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # 注册适应度函数
    toolbox.register("evaluate", evalSupply, supplies=supplies)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # 初始化种群
    population = toolbox.population(n=300)
    
    # 使用遗传算法进行优化
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)
    
    # 返回最优解
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual, sum(best_individual[i] * supplies[i] for i in range(len(supplies)))

# 运行遗传算法
best_individual, best_sum = genetic_algorithm(supplies)

# 打印结果
print("最优解：", best_individual)
print("最优解的总供货量：", best_sum)
print("使用的商家数量：", sum(best_individual))


d:\Anaconda\envs\new_env\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\Anaconda\envs\new_env\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	300   
1  	185   
2  	177   
3  	170   
4  	194   
5  	191   
6  	181   
7  	195   
8  	181   
9  	178   
10 	199   
11 	183   
12 	183   
13 	180   
14 	182   
15 	182   
16 	174   
17 	166   
18 	184   
19 	190   
20 	199   
21 	190   
22 	169   
23 	167   
24 	168   
25 	184   
26 	189   
27 	167   
28 	175   
29 	180   
30 	150   
31 	190   
32 	199   
33 	193   
34 	187   
35 	169   
36 	185   
37 	187   
38 	181   
39 	182   
40 	185   
41 	185   
42 	193   
43 	171   
44 	177   
45 	185   
46 	195   
47 	180   
48 	181   
49 	173   
50 	165   
最优解： [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
最优解的总供货量： 6206260.732323231
使用的商家数量： 35


处理数据,取出35家商家240周的最大值

In [24]:
# 将 selection_list 转换为布尔掩码
selection_mask = pd.Series(best_individual, dtype=bool)

# 使用布尔掩码提取数据
selected_rows = data[selection_mask]
selected_num = selected_rows['商家编号'].tolist()
# 根据商家编号筛选出第二个表格中的对应行
selected_sales = sales[sales['供应商ID'].isin(selected_num)]

# 按行取最小值，结果会是一个 Series
max_values = selected_sales.iloc[:, 2:].max(axis=1)

selected_sales['最大值'] = max_values

selected_sales

C:\Users\海棠微雨\AppData\Local\Temp\ipykernel_18128\1682130814.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_sales['最大值'] = max_values


,供应商ID,材料分类,W001,W002,W003,W004,W005,W006,W007,W008,...,W232,W233,W234,W235,W236,W237,W238,W239,W240,最大值
30,S031,B,5,84,69,79,97,103,84,102,...,169,166,165,143,191,162,180,164,193,251
36,S037,C,0,0,0,0,0,0,0,0,...,10,9,4,3,14,21,1,1,0,5398
39,S040,B,57,111,90,283,116,106,90,113,...,114,125,101,108,98,101,97,107,115,432
54,S055,B,55,812,56,51,74,59,56,69,...,56,57,59,51,50,57,48,70,59,1216
79,S080,C,55,21,89,17,104,24,82,27,...,31,146,31,141,23,126,32,105,25,215
107,S108,B,556,7871,832,636,761,707,615,636,...,660,804,703,668,639,725,636,755,698,7885
125,S126,C,5,0,0,0,0,0,0,0,...,0,0,11,0,0,0,0,0,0,15114
130,S131,B,2,539,657,628,620,663,426,661,...,619,596,705,576,716,684,608,599,660,1014
138,S139,B,6390,0,7023,0,5,16,10,11,...,5,17,7,15,6,10,13,9,4,10207
139,S140,B,10,0,0,0,15687,4,1,11,...,9,8,3,4,3,5,4,2,5,21293


In [27]:
# 创建DataFrame保存结果
result_df = pd.DataFrame(
    selected_sales.iloc[:, [0,1,-1]]
)

# 输出到Excel文件
output_path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/240周35家商家的供货量最大值.xlsx"
result_df.to_excel(output_path, index=False)

print(f"由大到小综合评价的索引已保存到 {output_path}")

由大到小综合评价的索引已保存到 C:/Users/海棠微雨/Desktop/cumcm2021c/C/240周35家商家的供货量最大值.xlsx
